##Ultralytics YOLOv3 설치

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!git clone https://github.com/ultralytics/yolov3
!cd yolov3;pip install -qr requirements.txt

In [ ]:
import torch
from IPython.display import Image, clear_output

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'You are using a CPU now'})")

##Oxford-IIIT Pet Dataset 다운로드
* Oxford Pet 데이터를 다운로드 후 /content/data 디렉토리에 압축 해제.

In [ ]:
!wget https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
!wget https://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz

In [ ]:
!mkdir /content/data
!tar -xvf images.tar.gz -C /content/data
!tar -xvf annotations.tar.gz -C /content/data

##Oxford-IIIT Pet Dataset Handling
* Pandas Dataframe으로 메타데이터 생성
* 데이터셋 review
* Train/Validation 데이터셋으로 분리


In [6]:
!mkdir /content/ox_pet;
!cd /content/ox_pet; mkdir images; mkdir labels;
!cd /content/ox_pet/images; mkdir train; mkdir val
!cd /content/ox_pet/labels; mkdir train; mkdir val

In [ ]:
import pandas as pd 

pd.read_csv('/content/data/annotations/trainval.txt', sep=' ', header=None, names=['img_name', 'class_id', 'etc1', 'etc2'])

In [8]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

def make_train_valid_df(list_filepath, img_dir, anno_dir, test_size=0.1):
  pet_df = pd.read_csv(list_filepath, sep=' ', header=None, names=['img_name', 'class_id', 'etc1', 'etc2'])
  pet_df['class_name'] = pet_df['img_name'].apply(lambda x:x[:x.rfind('_')])
  
  pet_df['img_filepath'] = img_dir + pet_df['img_name']+'.jpg'
  pet_df['anno_filepath'] = anno_dir + pet_df['img_name']+'.xml'
  pet_df = remove_no_annos(pet_df)

  train_df, val_df = train_test_split(pet_df, test_size=test_size, stratify=pet_df['class_id'], random_state=8986)
  return pet_df, train_df, val_df

def remove_no_annos(df):
  remove_rows = []
  for index, row in df.iterrows():
    anno_filepath = row['anno_filepath']
    if not os.path.exists(anno_filepath):
      print('##### index:', index, anno_filepath, '가 존재하지 않아서 Dataframe에서 삭제함')
      remove_rows.append(index)
  df = df.drop(remove_rows, axis=0, inplace=False)
  return df
  

In [ ]:
pet_df, train_df, val_df = make_train_valid_df('/content/data/annotations/trainval.txt', 
                                               '/content/data/images/', '/content/data/annotations/xmls/', test_size=0.1)

In [ ]:
pet_df.head()

In [11]:
import os

for index, row in pet_df.iterrows():
  anno_filepath = row['anno_filepath']
  if not os.path.exists(anno_filepath):
    print(anno_filepath)

## Oxford Pet 데이터 세트의 annotation을 Ultralytics Yolo format으로 생성
* annotation용 xml 파일을 txt 파일로 변환
* 하나의 이미지마다 하나의 txt 파일을 할당
* 하나의 xml annotation 파일을 Yolo 포맷용 txt 파일로 변환하는 함수 생성
* voc annotation의 좌상단(Top left: x1, y1), 우하단(Bottom right: x2, y2) 좌표를 Bounding Box 중심 좌표(Center_x, Center_y)와 너비(width), 높이(height)로 변경
*  모든 값 위치정보를 0~1 사이 값으로 변환
* class_id는 여러개의 label들을 '0' 부터 순차적으로 부여

In [ ]:
CLASS_NAMES = pet_df['class_name'].unique().tolist()
print(CLASS_NAMES)

In [13]:
import glob
import xml.etree.ElementTree as ET

def xml_to_txt(input_xml_file, output_txt_file, object_name):
  tree = ET.parse(input_xml_file)
  root = tree.getroot()
  img_node = root.find('size')
  if img_node is None:
    return None 
  img_width = int(img_node.find('width').text)
  img_height = int(img_node.find('height').text)

  value_str = None
  with open(output_txt_file, 'w') as output_fpointer:
    for obj in root.findall('object'):
        xmlbox = obj.find('bndbox')
        x1 = int(xmlbox.find('xmin').text)
        y1 = int(xmlbox.find('ymin').text)
        x2 = int(xmlbox.find('xmax').text)
        y2 = int(xmlbox.find('ymax').text)
        if (x1 < 0) or (x2 < 0) or (y1 < 0) or (y2 < 0):
          break
        class_id, cx_norm, cy_norm, w_norm, h_norm = convert_yolo_coord(object_name, img_width, img_height, x1, y1, x2, y2)
        value_str = ('{0} {1} {2} {3} {4}').format(class_id, cx_norm, cy_norm, w_norm, h_norm)
        output_fpointer.write(value_str+'\n')

def convert_yolo_coord(object_name, img_width, img_height, x1, y1, x2, y2):
  class_id = CLASS_NAMES.index(object_name)
  center_x = (x1 + x2)/2
  center_y = (y1 + y2)/2
  width = x2 - x1
  height = y2 - y1
  center_x_norm = center_x / img_width
  center_y_norm = center_y / img_height
  width_norm = width / img_width
  height_norm = height / img_height

  return class_id, round(center_x_norm, 7), round(center_y_norm, 7), round(width_norm, 7), round(height_norm, 7)
  

In [ ]:
 class_id = CLASS_NAMES.index('yorkshire_terrier')
 print(class_id)

In [15]:
xml_to_txt('/content/data/annotations/xmls/Abyssinian_1.xml', '/content/ox_pet/labels/train/Abyssinian_1.txt', 'Abyssinian')

## VOC Format의 annotation 파일을 Yolo format으로 변환 후 Ultralytics directory 구조로 입력
* VOC format의 XML 파일을 Yolo format으로 변환
* Yolo format의 txt파일이 저장될 디렉토리를 생성

In [16]:
import shutil

def make_yolo_anno_file(df, tgt_images_dir, tgt_labels_dir):
  for index, row in df.iterrows():
    src_image_path = row['img_filepath']
    src_label_path = row['anno_filepath'] 
    object_name = row['class_name']
    target_label_path = tgt_labels_dir + row['img_name']+'.txt'
    shutil.copy(src_image_path, tgt_images_dir)
    xml_to_txt(src_label_path, target_label_path, object_name)


In [17]:
make_yolo_anno_file(train_df, '/content/ox_pet/images/train/', '/content/ox_pet/labels/train/') 
make_yolo_anno_file(val_df, '/content/ox_pet/images/val/', '/content/ox_pet/labels/val/')

## Oxford Pet Dataset 학습 수행
* 생성된 Directory 구조에 맞춰서 dataset용 yaml 파일 생성
* 학습 인자를 설정한 후 학습 수행

In [18]:
!cp /content/drive/MyDrive/ultra_yolov3/ox_pet.yaml /content/ox_pet/ox_pet.yaml

In [ ]:
# 30 epoch train시 약 1시간 정도 소요.
# Colab은 활동이 없으면 time limit을 걸기 때문에 browser console에서 매크로 설정 필요
# F12 또는 shift+cntr+i 로 browser console 열기 

'''
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect,60000)
'''

In [ ]:
!mkdir "/content/drive/MyDrive/ultra_yolov3"

In [ ]:
!cd /content/yolov3; python train.py --img 640 --batch 16 --epochs 100 --data /content/ox_pet/ox_pet.yaml --weights yolov3.pt --project=/content/drive/MyDrive/ultra_yolov3 \
                                     --name oxpet --exist-ok

## 학습된 모델 파일을 이용하여 image Inference 수행
* 이미지 파일로 inference 수행

In [ ]:
!cd yolov3;python detect.py --source /content/drive/MyDrive/ultra_yolov3/Pug_1.jpg --weights /content/drive/MyDrive/ultra_yolov3/oxpet/weights/best.pt --conf 0.4 \
                            --project=/content/data/output --name=run_image --exist-ok --line-thickness 2

In [20]:
from IPython.display import Image, clear_output 

In [ ]:
Image(filename='/content/data/output/run_image/Pug_1.jpg', width=400)

In [ ]:
Image(filename='/content/data/output/run_image/Cookie.jpg', width=400)

##Test Data Evalutation

In [ ]:
!pip install terminaltables

In [ ]:
!cd yolov3; python val.py --weights /content/drive/MyDrive/ultra_yolov3/oxpet/weights/best.pt  --data /content/ox_pet/ox_pet.yaml \
                           --project /content/data/output --name=test_result --exist-ok --img 640 --iou 0.65

In [ ]:
Image(filename='/content/data/output/test_result/confusion_matrix.png', width=800)